In [1]:
!pip install -q geopandas;
!pip install -q geemap;
!pip install -q --upgrade folium;

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.2 MB/s eta 0:00:00


In [2]:
import json
import ee, datetime, folium
import pandas as pd
from IPython.display import Image
from pylab import *
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from folium.plugins import HeatMap
import io
import fiona
import branca.colormap as cm
import geemap
from shapely.geometry import shape
from shapely.geometry import shape, Polygon, MultiPolygon
import os

# Extract Satellite Indices per Geographic Area in England

## First authenticate GEE

In [3]:
# Trigger the authentication flow.
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=2_oMIcF9h3QFBOcI2qOxKrLuewRwkrxdIi87H0Su9V8&tc=CoJvVQdvYGsNBuKnaG4TU--iL0zmR95DTmyx18rTAco&cc=KweQ9iMqKGSMN574uM5aGuL4c_ym2U3SKT7sdwy1PFo

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BX8K7SSI-e8nkMMYVHhF-Uyke0Pxfd-1JnsdCvJ4RYDSfwsziGfHHI

Successfully saved authorization token.


In [6]:
# Initialize the library.
PROJECT_NAME = 'ee-env-data-extractor' ## TODO: CHANGE TO YOUR PROJECT NAME

ee.Initialize(project=PROJECT_NAME)

## Then also connect to Google Drive


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Global Variables

In [7]:
your_location = 'ENV_DATA_EXTRACTOR' ## TODO: CHANGE TO YOUR GDRIVE LOCATION
project_folder = f"drive/MyDrive/{your_location}/environmental_data_extractor/Colab_Notebooks/"

In [8]:
DynamicWorld_CLASS_NAMES = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops',
    'shrub_and_scrub', 'built', 'bare', 'snow_and_ice'];

SELECTORS = {
    # https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v200#bands
    'ESA-WorldCover' : ['Map'],
    # https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_DYNAMICWORLD_V1#bands
    'GOOGLE-DynamicWorld' : DynamicWorld_CLASS_NAMES,
}


SCALE = {
    'ESA-WorldCover' :  10,
    'GOOGLE-DynamicWorld':100,

}

SAMPLE_SIZE = {
    'ESA-WorldCover' :  None,
    'GOOGLE-DynamicWorld': 72,

}

## 0 Functions

In [9]:
# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [10]:
# Define a function for displaying shapefiles saved into a feature collection
def plot_ee_feature_shapefile_folium(ee_feature, zoom_start=7):
  # Get the bounds of the feature collection
  aoi = ee_feature.geometry()

  # Create a folium map object.
  center = aoi.centroid(10).coordinates().reverse().getInfo()
  m = folium.Map(location=center, zoom_start=zoom_start)

  # Add layers to the folium map.
  # Add the feature collection to the map as a GeoJSON layer
  m.add_child(folium.GeoJson(data=ee_feature.getInfo()))

  # Add a layer control panel to the map.
  m.add_child(folium.LayerControl())

  # Display the map.
  display(m)

In [11]:
def plot_ee_feature_shapefile_geemap(fc, zoom_start=6, coordinates=(-0.1278, 51.5074), color=None):

  # Create a Map object
  Map = geemap.Map() # set initial center and zoom level

  # Add the Feature Collection to the Map, centered on the shapefile
  Map.addLayer(fc, {}, 'Shapefile')

  Map.centerObject(fc, zoom_start)

  if color:
    # Add a color based on the 'means' property
    colorized = fc.map(lambda feature: feature.set({'style': {'color': 'blue', 'fillColor': feature.get(color)}}))

  # Add the colorized feature collection to the map
  Map.addLayer(colorized, {}, 'Colorized')

  display(Map)

In [12]:
def spatial_join_ee_fc(ee_fc_fine, ee_fc_coarse):

  # Define the spatial join relationship
  spatialFilter = ee.Filter.intersects(
    leftField='.geo', # geometry field in regions feature collection
    rightField='.geo', # geometry field in wards feature collection
    maxError=10) # tolerance distance in meters

  # Perform the spatial join using the defined spatial filter
  ee_fc_joined = ee.Join.simple().apply(
    primary=ee_fc_fine,
    secondary=ee_fc_coarse,
    condition=spatialFilter)

  return ee_fc_joined

In [13]:
def get_satellite_imagery(ee_the_region, sat, START_DATE, END_DATE):

  params = SELECTORS[sat]

  # these are parameters for selecting the Sentinel Image
  aoi = ee_the_region.geometry()

  if sat == 'ESA-WorldCover':

    # Load the dataset
    im_dat = ee.ImageCollection('ESA/WorldCover/v200')

    # Filter data by date and location
    filtered = im_dat.filterDate(START_DATE, END_DATE).filterBounds(aoi)

    # Calculate mean
    mean_values = filtered.select(params).first()


  elif sat == 'GOOGLE-DynamicWorld':

    # Load the dataset
    im_dat = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')

    # Filter data by date and location
    filtered = im_dat.filterDate(START_DATE, END_DATE).filterBounds(aoi)

    # Get the count of images in the filtered collection
    image_count = filtered.size().getInfo()
    # Print the count of images
    print("Number of images in the filtered DYNAMICWORLD collection:", image_count)


    # Define the number of images to sample
    sample_size = SAMPLE_SIZE[sat]  # Adjust this value as needed
    print("Number of images you are asking in your sample:", SAMPLE_SIZE[sat] )

    # Randomly select a subset of images
    sampled_images = ee.ImageCollection(filtered.randomColumn('random').sort('random').limit(sample_size))
    # Calculate mean
    mean_values = sampled_images.select(params).mean()

    # # Calculate mean
    # mean_values = filtered.select(params).mean()


  return mean_values

In [14]:
def efficent_fc_to_gdf(fc, selectors, batch_size=2000):

  print ("running efficient conversion to gdf")

  # Get the total number of features in the feature collection
  num_features = fc.size().getInfo()

  # Define a list to store the GeoDataFrames
  gdfs = []

  # Loop over the feature collection in batches of size batch_size
  for i in range(0, num_features, batch_size):
      print ("running efficient conversion to gdf. data {} of {}...".\
              format(i, int(num_features)))
      # Get the next batch of features using the limit() method
      # features = fc.limit(batch_size, str(i))
      features_list = fc.toList(batch_size, i)
      features = ee.FeatureCollection(features_list)

      # Convert the batch of features to a GeoDataFrame
      gdf = geemap.ee_to_geopandas(features, selectors=selectors)

      # Add the GeoDataFrame to the list
      gdfs.append(gdf)

  # Concatenate the GeoDataFrames into a single GeoDataFrame
  final_gdf = pd.concat(gdfs)

  # Display the GeoDataFrame
  # print(final_gdf.head())

  return final_gdf

In [19]:
def extract_lsoa_scores_region(region_name, ee_regions, ee_lsoas_UK, sat, START_DATE, END_DATE):

  year = START_DATE.split("-")[0]
  print("****** Processing {} in {} year *******".format(region_name, year))

  # prepare region geographies
  ee_the_region = ee_regions.filter(ee.Filter.eq('RGN22NM',region_name))
  ee_region_lsoas = spatial_join_ee_fc(ee_lsoas_UK, ee_the_region)
  print ("prepared Region geographies ...")

  # pull satellite imagery
  sat_data = get_satellite_imagery(ee_the_region, sat, START_DATE, END_DATE)
  print ("pulled imagery ...")

  # # Get the metadata of the feature
  # metadata = sat_data.getInfo()
  # # Print the metadata
  # print(metadata)

  # get the shapefile we need
  shapefile = ee_region_lsoas

  # Get the mean value of each band within the shapefile
  # this one works in the CRS of the first image band; mainly it is for our data
  # EPSG:4326 -- the WGS84 geographic coordinate reference system, which has its units in degrees.
  # means = sat_data.reduceRegions(collection=shapefile, reducer=ee.Reducer.frequencyHistogram(), scale=SCALE[sat])
  means = sat_data.reduceRegions(collection=shapefile, reducer=ee.Reducer.mean(), scale=SCALE[sat])
  print ("calculated LSOAs scores ...")

  # # # Get the first feature in the collection
  first_feature = means.first()
  # # Get the metadata of the feature
  metadata = first_feature.getInfo()
  # # Print the metadata
  print(metadata)

  if sat == 'ESA-WorldCover':

    means = sat_data.reduceRegions(collection=shapefile, reducer=ee.Reducer.frequencyHistogram(), scale=SCALE[sat])

    def add_normalized_histogram(feature):
        # Get the histogram for the feature
        histogram = feature.get('histogram')
        # Calculate the normalized histogram
        dict = ee.Dictionary(histogram)
        sum = ee.Array(dict.values()).reduce(ee.Reducer.sum(),[0]).get([0])
        new_dict = dict.map(lambda k, v: ee.Number(v).divide(sum).multiply(100))
        # Add the normalized histogram to the feature as a new property
        return feature.set('normalized_histogram', new_dict)

    old_means = means

    # Map the function over the feature collection and get a new feature collection with the new property
    means = old_means.map(add_normalized_histogram)

  # a small hack because with 1 param, this gives a column means
  # but with two and more keeps their original names
  if len(SELECTORS[sat]) == 1:
    selectors = ['LSOA21NM', 'LSOA21CD', 'mean']
    if sat == 'ESA-WorldCover':
        selectors = ['LSOA21NM', 'LSOA21CD', 'normalized_histogram', 'histogram']
  else:
    selectors = ['LSOA21NM', 'LSOA21CD'] + SELECTORS[sat]
  # Convert the feature collection to a GeoPandas dataframe
  # gdf_means = geemap.ee_to_geopandas(means, selectors=selectors)
  gdf_means = efficent_fc_to_gdf(means, selectors=selectors)
  # # Define the CRS of the GeoJSON data
  # gdf_means.crs = 'EPSG:4326'
  print ("converted feature collection to geopandas df ...")

  # save csv with results
  csv_results_folder = project_folder + \
         "../results/{}/lsoa/{}/".format(sat, year)

  if not os.path.exists(csv_results_folder):
      os.makedirs(csv_results_folder)

  # save csv with results
  csv_results_address = csv_results_folder + f"{region_name}.csv"

  # create a Pandas dataframe with selected columns
  df_means = pd.DataFrame(gdf_means[selectors])
  df_means.to_csv(csv_results_address, index=False)
  print ("saved csv ...")

  return gdf_means

## missing values

In [16]:
def check_missing_values(folder_path):
    files_with_missing_values = []
    files_without_missing_values = []

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            df = pd.read_csv(file_path)
            if df.isnull().values.any():
                files_with_missing_values.append(file_name)
            else:
                files_without_missing_values.append(file_name)

    return files_without_missing_values, files_with_missing_values


## 1 Download / Read In Data

## Spatial Data

### Region Data

In [17]:
# England regions data is from here https://geoportal.statistics.gov.uk/
# https://geoportal.statistics.gov.uk/datasets/ons::regions-december-2022-en-bfc/explore?location=52.916331%2C-2.000000%2C7.74

# drive_address = project_folder + "data/England-regions/Regions_(December_2022)_EN_BFC.shp"
asset_address = f"projects/{PROJECT_NAME}/assets/regions"

# 2 we can read the data as a feature shapefile from GEE assets
ee_regions = ee.FeatureCollection(asset_address)

### LSOAs Data

In [18]:
# UK LSOAs data is from here https://geoportal.statistics.gov.uk/
# https://geoportal.statistics.gov.uk/datasets/766da1380a3544c5a7ca9131dfd4acb6_0/explore?location=51.420294%2C-0.114413%2C11.00

# drive_address = project_folder + "data/UK-LSOAs/LSOA_(Dec_2021)_Boundaries_Generalised_Clipped_EW_(BGC).shp"
asset_address = f"projects/{PROJECT_NAME}/assets/lsoas"

# 2 we can read the data as a feature shapefile from GEE assets
ee_lsoas_UK = ee.FeatureCollection(asset_address)

# Process an LSOA

In [21]:
lsoa_NDVI_scores = extract_lsoa_scores_region(region_name='London',\
                            ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
                            sat='GOOGLE-DynamicWorld', START_DATE='2021-01-01', END_DATE='2022-01-02')

****** Processing London in 2021 year *******
prepared Region geographies ...
Number of images in the filtered DYNAMICWORLD collection: 207
Number of images you are asking in your sample: 72
pulled imagery ...
calculated LSOAs scores ...
{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[0.2482462848062044, 51.50447284055022], [0.249138555903767, 51.505013793744325], [0.24961544583117296, 51.506272583021534], [0.25023906198828305, 51.506006127526135], [0.2510027588109454, 51.506406687010546], [0.25150320587348535, 51.50596405453736], [0.25218716259493135, 51.50590621183805], [0.25384180776456877, 51.506383431552024], [0.25378710191900566, 51.50701390112902], [0.2543068603063429, 51.50696377760156], [0.2544879528211024, 51.50758981323938], [0.2550508433719112, 51.50754320159254], [0.2554401851151281, 51.50890638552053], [0.2554483929992349, 51.50892850291979], [0.25692335599618493, 51.50880635349742], [0.2589264937473455, 51.50909372876333], [0.26100416747616506, 51.5

# Process all LSOAs

## 2020

### after we ran the code below once, we will check whether all regions / LSOAs got results (due to clouds, some LSOAs might have not got any data points for the estimate)

In [ ]:
sat = "GOOGLE-DynamicWorld"
year = "2020"
SAMPLE_SIZE[sat] = 72
SCALE[sat] = 100

# for region_name in regions['RGN22NM'].unique():
#     lsoa_LC_scores = extract_lsoa_scores_region(region_name=region_name, ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
#                       sat='GOOGLE-DynamicWorld', START_DATE='2020-01-01', END_DATE='2021-01-01')

In [ ]:
sat = "GOOGLE-DynamicWorld"
year = "2020"

# Provide the path to the input folder
input_folder = project_folder + "results/{}/lsoa/{}/".format(sat, year)

# Call the function to check missing values in CSV files
files_without_missing, files_with_missing = check_missing_values(input_folder)

# Print the results
print("Files without missing values:")
for file_name in files_without_missing:
    print(file_name)

print("\nFiles with missing values:")
for file_name in files_with_missing:
    print(file_name)

Files without missing values:
North East.csv
Yorkshire and The Humber.csv
East Midlands.csv
West Midlands.csv
East of England.csv
London.csv
South East.csv
South West.csv
North West.csv
whole_England_land_cover.csv

Files with missing values:


### if we have some LSOAs in some regions missing; run the code below only on them (by increasing and tweaking samle size and increasing scale, until we are happy)

In [ ]:
SAMPLE_SIZE[sat] = 144
SCALE[sat] = 200

lsoa_scores = extract_lsoa_scores_region(region_name="North West", ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
                      sat='GOOGLE-DynamicWorld', START_DATE='2020-01-01', END_DATE='2021-01-01')

****** Processing North West in 2020 year *******
prepared Region geographies ...
Number of images in the filtered DYNAMICWORLD collection: 332
Number of images you are asking in your sample: 144
pulled imagery ...
calculated LSOAs scores ...
{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-2.552834407880523, 52.88904846152916], [-2.550879249194629, 52.88963769757437], [-2.550674581375136, 52.8903704992581], [-2.549469846689583, 52.89138026381314], [-2.5368826394260413, 52.88368605502287], [-2.5359902574721294, 52.88430596241071], [-2.535700748930522, 52.88476247204999], [-2.5360426334314945, 52.884966153669154], [-2.5353545497325234, 52.8854066004688], [-2.5342910644854837, 52.88531767114396], [-2.5341191247679458, 52.88573352908369], [-2.5329295309369377, 52.88618955676338], [-2.5325500678227075, 52.88592796428037], [-2.5322448696832622, 52.8863131749779], [-2.529353382758226, 52.88636623783194], [-2.5289329970244747, 52.88759090666669], [-2.5281538834643116, 52

## 2019

In [ ]:
sat = "GOOGLE-DynamicWorld"
year = "2019"
SAMPLE_SIZE[sat] = 72
SCALE[sat] = 100

for region_name in regions['RGN22NM'].unique():
    lsoa_LC_scores = extract_lsoa_scores_region(region_name=region_name, ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
                      sat='GOOGLE-DynamicWorld', START_DATE='2019-01-01', END_DATE='2020-01-01')

****** Processing North East in 2019 year *******
prepared Region geographies ...
pulled imagery ...
calculated LSOAs scores ...
{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-1.483891824255598, 54.86625184516661], [-1.483141782538926, 54.865026265085284], [-1.4817217347520169, 54.86538607120365], [-1.4820246414411484, 54.86407573161617], [-1.4812126118110736, 54.86346549964107], [-1.4797489627425593, 54.86301765228609], [-1.4794159061042171, 54.86244960924335], [-1.478446301306342, 54.86267293483722], [-1.4773320828603522, 54.862110513937196], [-1.4759165666953848, 54.862122231407966], [-1.47571335760535, 54.861099729867284], [-1.4745706668865355, 54.859163074387624], [-1.473808092677857, 54.8583389368168], [-1.4720135657245104, 54.85896016704654], [-1.4714157712698157, 54.8585471834146], [-1.470849596174542, 54.858669614971696], [-1.4651530191429734, 54.859617201857624], [-1.462315258247639, 54.85958233102275], [-1.4614400699554808, 54.85793676073699], [-1.453

In [ ]:
sat = "GOOGLE-DynamicWorld"
year = "2019"

# Provide the path to the input folder
input_folder = project_folder + "results/{}/lsoa/{}/".format(sat, year)

# Call the function to check missing values in CSV files
files_without_missing, files_with_missing = check_missing_values(input_folder)

# Print the results
print("Files without missing values:")
for file_name in files_without_missing:
    print(file_name)

print("\nFiles with missing values:")
for file_name in files_with_missing:
    print(file_name)

Files without missing values:
North East.csv
West Midlands.csv
East of England.csv
London.csv
South East.csv
North West.csv
Yorkshire and The Humber.csv
East Midlands.csv
South West.csv

Files with missing values:


### if we have some LSOAs in some regions missing; run the code below only on them (by increasing and tweaking samle size and increasing scale, until we are happy)

In [ ]:
SAMPLE_SIZE[sat] = 288
SCALE[sat] = 200

for region_name in files_with_missing:
    region_name = region_name.split(".")[0]
    lsoa_LC_scores = extract_lsoa_scores_region(region_name=region_name, ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
                      sat='GOOGLE-DynamicWorld', START_DATE='2019-01-01', END_DATE='2020-01-01')

****** Processing South West in 2019 year *******
prepared Region geographies ...
Number of images in the filtered DYNAMICWORLD collection: 651
Number of images you are asking in your sample: 288
pulled imagery ...
calculated LSOAs scores ...
{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-1.5996308547869926, 51.68907425722547], [-1.5969962469935053, 51.68815016168121], [-1.5927168563865868, 51.690469869390384], [-1.5902609470436595, 51.69123803664412], [-1.586647396444157, 51.69292150032555], [-1.58408588613705, 51.6934302718114], [-1.5832399298105604, 51.69326210088744], [-1.583157029921696, 51.69369036508038], [-1.5835807828328028, 51.693687402581865], [-1.5825101675062627, 51.69529490187084], [-1.5786866419207826, 51.69633473125697], [-1.5774514764180256, 51.69707119613175], [-1.5776811309859382, 51.697866557842964], [-1.5755512618751308, 51.69980058135148], [-1.5734200585009186, 51.7003195788994], [-1.573286143316297, 51.700818956078656], [-1.572386953992006

## 2021

In [ ]:
# for region_name in regions['RGN22NM'].unique():
#     lsoa_LC_scores = extract_lsoa_scores_region(region_name=region_name, ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
#                       sat='GOOGLE-DynamicWorld', START_DATE='2021-01-01', END_DATE='2022-01-01')

In [ ]:
sat = "GOOGLE-DynamicWorld"
year = "2021"

# Provide the path to the input folder
input_folder = project_folder + "results/{}/lsoa/{}/".format(sat, year)

# Call the function to check missing values in CSV files
files_without_missing, files_with_missing = check_missing_values(input_folder)

# Print the results
print("Files without missing values:")
for file_name in files_without_missing:
    print(file_name)

print("\nFiles with missing values:")
for file_name in files_with_missing:
    print(file_name)

Files without missing values:
North East.csv
North West.csv
Yorkshire and The Humber.csv
East Midlands.csv
West Midlands.csv
East of England.csv
London.csv
South East.csv

Files with missing values:
South West.csv


In [ ]:
SAMPLE_SIZE[sat] = 176
SCALE[sat] = 300

for region_name in files_with_missing:
    region_name = region_name.split(".")[0]
    lsoa_LC_scores = extract_lsoa_scores_region(region_name=region_name, ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
                      sat='GOOGLE-DynamicWorld', START_DATE='2021-01-01', END_DATE='2022-01-01')

****** Processing South West in 2021 year *******
prepared Region geographies ...
Number of images in the filtered DYNAMICWORLD collection: 622
Number of images you are asking in your sample: 176
pulled imagery ...
calculated LSOAs scores ...
{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-1.5996308547869926, 51.68907425722547], [-1.5969962469935053, 51.68815016168121], [-1.5927168563865868, 51.690469869390384], [-1.5902609470436595, 51.69123803664412], [-1.586647396444157, 51.69292150032555], [-1.58408588613705, 51.6934302718114], [-1.5832399298105604, 51.69326210088744], [-1.583157029921696, 51.69369036508038], [-1.5835807828328028, 51.693687402581865], [-1.5825101675062627, 51.69529490187084], [-1.5786866419207826, 51.69633473125697], [-1.5774514764180256, 51.69707119613175], [-1.5776811309859382, 51.697866557842964], [-1.5755512618751308, 51.69980058135148], [-1.5734200585009186, 51.7003195788994], [-1.573286143316297, 51.700818956078656], [-1.572386953992006